# **Calculating WER/CER of transcripts**

In this notebook accompanies the Medium blogpost **"Text analytics on Dutch cycling training podcasts, part I Evaluating speech-to-text methods"**. 

We will use the textual output of the four methods created in notebook **"Methods for transcription"** and simply calculate some performance (error) metrics, more specifically WER and CER. You can find the used transcription data files in this [folder](https://github.com/RuudVelo/medium/tree/main/blogpost_1/data)

In [1]:
# Install necessary packages

!pip install jiwer
!pip install num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=facfb0fe42adef3afc9c2b0af2c7b0b02887df75724145cc803106490886c5ec
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [2]:
# import libraries

import re
import os
import json
import numpy as np

from num2words import num2words

import jiwer
from jiwer import wer, cer

from google.colab import drive

In [3]:
# set Google drive and select folder
drive.mount('/content/drive')

os.chdir("drive/My Drive/Colab Notebooks/beter_worden_podcasts")

Mounted at /content/drive


In [4]:
# read in the files. Since they all have slightly different content they are loaded seperately

# read ground truth file
f = open("BW_PD_EP_54_SCHAKELEN_GROUND_TRUTH.json")
ground_truth = json.load(f)['text'] 

# read Youtube subtitles
f = open("youtube_subtitles.jsonl")
hypothesis_youtube = json.load(f) 

# read Vosk transcription
f = open("vosk_subtitles.jsonl")
hypothesis_vosk = json.load(f) 

# read Wav2Vec2 transcription
f = open("GroNLP_wav2vec2_dutch_large_ft_cgn_transcription.json")
hypothesis_wav2vec2 = json.load(f)['text']

# read Whisper transcription
f = open("BW_PD_EP_54_SCHAKELEN_WHISPER_LARGE_V1_TS.jsonl")
hypothesis_whisper = json.load(f) 
hypothesis_whisper = "".join([item['text'] for item in hypothesis_whisper])

In [9]:
# see some example text

ground_truth[:2000]

' Deze podcast maken we samen met Join.cc, de fietsapp voor alle wielrenners. Zin om te fietsen? Geen zin om te fietsen. Toch gaan. Jezelf op die fiets trekken. Regen, hitte, omhoog omlaag, zweten puffen, slechte poten, wonderbenen, genieten, kapot gaan, los trappen, bijtanken, douchen en door. Het leven van een renner gaat niet over rozen en al helemaal niet, als je jezelf wilt verbeteren. Maar hoe dan? Waar moet ik nou op letten als ik gericht beter wil worden? Als ik harder of verder wil gaan? Gewoon fitter wil worden en meer wil genieten op de fiets? We gaan het hebben over trainen. Eten. Slapen. Op zoek naar de kennis, maar vooral ook naar de tips en slimmigheidjes waar we morgen mee aan de slag kunnen. Je luistert naar de Beter Worden podcast van Live Slow Ride Fast. Rechtstreeks vanuit hotel Valkenburg in het mooie Zuid Limburg. Mijn naam is Stefan Bolt en tegenover mij zitten ze Laurens en Ten Dam en Jim van den Berg. Lau, als je naar je grote blad achter wil schakelen, welke h

In [10]:
# see some example text

hypothesis_youtube[:2000]

'deze podcast maken we samen met CC de fiets heb voor alle wielrenners toch gaan jezelf op die fiets trekken regen hitte omhoog omlaag zweet te puffen slechte poten wonder benen genieten kapot gaan los trappen bij tanken douchen en door het leven van Arena gaat niet over rozen en nou helemaal niet als je er heel veel verbeteren Maar hoe dan waar moet ik nou opletten als ik gericht beter wil worden als ik harder hoe verder wil gaan gewoon fitter wil worden en meer wil genieten op de fiets We gaan het hebben over trainen heten slapen zoek naar de kennisma vooral ook naar de tips en slimme Geitjes waar we morgen mee aan de slag kunnen je luistert naar de beter worden podcast van livslow Right Fast rechtstreeks vanuit hotel Valkenburg in het mooie zuid-limburg mijn naam is Stefan Bolt en tegenover mij zitten ze Laurens en dan en Jim van de Berg als je naar je grote plat achter wil schakelen Welke hendel pak je dan Thanks Peter Nee ik bedoel in jouw wereld is het groot maar bij mij is het g

In [11]:
# see some example text

hypothesis_vosk[:4000]

'  deze podcast maken we samen met joint punt c de fietsen heb voor alle wielrenners twee uur het totaal niet baten en zin om te fietsen geen zin om te fietsen toch gaan jezelf op de fiets trekken regen hitte omhoog omlaag zweten buffer slechte boten wonnen bijna geen niet de kapot gaan lossen trap wat bij denken douchen en door het leven van een renner gaat niet over rozen en al helemaal niet als je jezelf wil verbeteren maar hoe dan waar moet ik nou op letten als ik gericht beter wil worden als ik harde verder wil gaan gewoon fitter wil worden en nu genieten op de fiets gaan het hebben over trainer eten slapen zoek naar de kennis maar vooral ook naar de tips en slimmigheidje is lange morgen mee aan de slag kunnen u luistert naar de beter worden podcast van lichtsnelheid vereist rechtstreeks vanuit hotel valkenburg in een mooie zuid-limburg mijn naam is een boot en tegenover mij zitten ze laurens ten dam en team van de berg ja blauw als je naar je grote plat achter schakelen welke hen

In [19]:
# see some example text

hypothesis_wav2vec2[:2500].lower()

"deze potkaast maken we samen met joyn puntcc de fietsep voor alle wielrenners tepre werv tel w telv nyv wetv tre env otv zin om te fietsen geen zin om te fietsen toch gaan jezelf op die fiets trekken regen hitte omhoog omlaag zweet te puffen slechte pote wonderbenen geniet te kapot gaan lostrappen bijtankendouche en door 't leven van een renner gaat niet over rose en nou helemaal niet als je jezelf wil verbeteren maar hoe dan wat moet ik nou opletten als ik gericht beter wil worden als ik harder of verder wil gaan gewoon fitter wil worden en meer wil genieten op de fiets we gaan 't hebben over trainen heten slapen zoek naar de kennis maar vooral ook naar de tips en slimmigheidjes waar we morgen mee aan de slag kunnen je luistert naar de betuwordepotcast van lipslow rit fest rechtstreeks vanuit hotel valkenburg in 't mooie zuid-limburg mijn names stefenbold en tegenover mij zit te sub lauensen dan en jim van denberg   uh   i  niet niet lau als je naar je grote blad achter wil schakelen

In [13]:
# see some example text

hypothesis_whisper[:2500]

" Deze podcast maken we samen met Join.cc, de fietsapp voor alle wielrenners. Zin om te fietsen? Geen zin om te fietsen. Toch gaan. Jezelf op die fiets trekken. Regen, hitte, omhoog omlaag, zweet te puffen, slechte poten, wonderbenen, genieten, kapotgaan, los trappen, bijtanken, douchen en door. Het leven van een renner gaat niet over rozen en nou helemaal niet, als je jezelf wil verbeteren. Maar hoe dan? Waar moet ik nou op letten als ik gericht beter wil worden? Als ik harder of verder wil gaan? Gewoon fitter wil worden en meer wil genieten op de fiets? We gaan het hebben over trainen, eten, slapen. Ik ben Steven Bolt en tegenover mij zitten ze Laurence St. Dan en Jim van den Berg. Lau, als je naar je grote blad achter wilt schakelen, welke hendel pak je dan? Links. Ja, die is pal spelen. Naar je grote blad achter, dat kan helemaal niet man. Of naar je... Kleine blad voor. Nee, ik bedoel naar je spaken. Als je naar je spaken toe is. Dat is toch je grote blad achter? Ja, nee. Grote en

In [5]:
# Define function to convert numbers to words

def text_num2words(text):
  list_string = [text]
  output_string = [re.sub('(\d+)', lambda m: num2words(m.group(), lang='nl'), sentence) for sentence in list_string]
  output_string = ''.join(output_string).strip()
  return output_string

In [6]:
# Invoke the function to convert numbers to words in the transcriptions

ground_truth_num2words = text_num2words(ground_truth)
hypothesis_youtube_num2words = text_num2words(hypothesis_youtube)
hypothesis_vosk_num2words = text_num2words(hypothesis_vosk) 
hypothesis_wav2vec2_num2words = text_num2words(hypothesis_wav2vec2)
hypothesis_whisper_num2words = text_num2words(hypothesis_whisper)

In [20]:
# Define function to calculate WER and CER

def wer_cer(ground_truth, sst_transcript):
  word_error = np.round(wer(ground_truth, sst_transcript),3)
  character_error = np.round(cer(ground_truth, sst_transcript),3)
  return print(word_error, 'wer'), print(character_error, 'cer')

In [23]:
# Invoke the functions per method

print("Youtube transcription:")
wer_cer(ground_truth, hypothesis_youtube_num2words)

print("Vosk transcription:")
wer_cer(ground_truth, hypothesis_vosk_num2words)

print("Wav2Vec2 transcription:")
wer_cer(ground_truth, hypothesis_wav2vec2_num2words)

print("Whisper transcription:")
wer_cer(ground_truth, hypothesis_whisper_num2words);

Youtube transcription:
0.592 wer
0.358 cer
Vosk transcription:
0.651 wer
0.388 cer
Wav2Vec2 transcription:
1.2 wer
0.955 cer
Whisper transcription:
0.261 wer
0.196 cer


In [9]:
# Use transformation to normalize WER

transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    jiwer.RemoveMultipleSpaces(),
    jiwer.ReduceToListOfListOfWords(word_delimiter=" ")
])

In [19]:
# Print the normalized WER per transcription method

print(np.round(jiwer.wer(
    ground_truth, 
    hypothesis_youtube, 
    truth_transform=transformation, 
    hypothesis_transform=transformation),3
),"Youtube transcription")

print(np.round(jiwer.wer(
    ground_truth, 
    hypothesis_vosk, 
    truth_transform=transformation, 
    hypothesis_transform=transformation),3
), "Vosk")

print(np.round(jiwer.wer(
    ground_truth, 
    hypothesis_wav2vec2, 
    truth_transform=transformation, 
    hypothesis_transform=transformation),3
),"Wav2Vec")

print(np.round(jiwer.wer(
    ground_truth, 
    hypothesis_whisper, 
    truth_transform=transformation, 
    hypothesis_transform=transformation),3
),"Whisper")

0.537 Youtube transcription
0.607 Vosk
0.636 Wav2Vec
0.227 Whisper
